This notebook applies morphological smoothing and reclassification using 2015 land cover map and external layers.

In [4]:
%matplotlib inline
import os
import datacube
import warnings
import numpy as np
import geopandas as gpd
import pandas as pd
import xarray as xr
import rioxarray
from rasterio.enums import Resampling
from datacube.utils.cog import write_cog
from deafrica_tools.spatial import xr_rasterize
from skimage.morphology import binary_dilation,disk
from skimage.filters.rank import modal
from odc.algo import xr_reproject
import matplotlib.pyplot as plt

output_crs='epsg:32735' # output crs: WGS84/UTM Zone 35S

# file paths
lesotho_tiles_shp='Data/Lesotho_boundaries_projected_epsg32735_tiles.shp' # Lesotho tiles shapefile
river_network_shp='Data/river_network.shp' # OSM river network data
road_network_shp='Data/road_network.shp' # OSM road network data
google_building_shp='Data/GoogleBuildingLayer.shp' # google bulding layer
road_network_shp='Data/road_network.shp' # OSM road network data
classification2015_raster='Data/classification2015.tif' # land cover map of 2015
classification2021_raster='Results/Land_cover2021_postproc_step2_mosaic.tif' # land cover map of 2021
hand_raster='Data/hand_Lesotho.tif' # Hydrologically adjusted elevations, i.e. height above the nearest drainage (hand)

# import Lesotho tiles and get bounding box
lesotho_tiles=gpd.read_file(lesotho_tiles_shp).to_crs(output_crs) # get bounding boxes of tiles covering Lesotho
tile_bboxes=lesotho_tiles.bounds

# load land cover maps
landcover2021=rioxarray.open_rasterio(classification2021_raster).astype(np.uint8).squeeze() # import land cover map of 2021
landcover2015=rioxarray.open_rasterio(classification2015_raster).astype(np.uint8).squeeze() # import land cover map of 2015
landcover2021=landcover2021.rio.reproject(output_crs) # reproject land cover map 2021
# remapping pixel values of 2015 land cover map to be consistent with 2021
original_class=[11, 12, 21, 22, 23, 24, 31, 33, 41, 42, 43, 44, 51, 61, 62, 71, 72, 73, 74, 75, 13, 14, 25, 32, 34, 35, 36, 37, 52]
mapped_class=[1, 1, 2, 2, 3, 14, 4, 5, 6, 6, 7, 8, 9, 10, 11, 12, 12, 12, 15, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0]
landcover2015_mapped=landcover2015.copy()
crs_copy_2015=landcover2015.rio.crs
for i in range(len(original_class)):
    landcover2015_mapped=xr.where(landcover2015==original_class[i],mapped_class[i],landcover2015_mapped)
del landcover2015
if landcover2015_mapped.rio.crs is None: # reassign crs which was lost during last step of using xr.where
    landcover2015_mapped.rio.write_crs(crs_copy_2015,inplace=True)
if landcover2015_mapped.rio.crs!=output_crs: # reproject 2015 land cover map if needed
    landcover2015_mapped=landcover2015_mapped.rio.reproject(resolution=10, dst_crs=output_crs,resampling=Resampling.nearest)

# load and pre-process external layers
road_network=gpd.read_file(road_network_shp).to_crs(output_crs) # import OSM road network data and reproject
road_network=road_network.loc[road_network['surface'].isin(['asphalt', 'paved', 'compacted', 'cobblestone', 
                                                             'concrete', 'metal', 'paving_stones', 
                                                             'paving_stones:30'])] # select road network by attributes
road_network.geometry=road_network.geometry.buffer(10) # buffer the road network by 10m
river_network=gpd.read_file(river_network_shp).to_crs(output_crs) # import OSM river network data and reproject
river_network=river_network.loc[river_network['waterway'].isin(['canal','river'])] # select river network by attribute
hand=xr.open_dataset(hand_raster,engine="rasterio").squeeze() # import hand layer
google_buildings=gpd.read_file(google_building_shp).to_crs(output_crs) # import google bulding layer
google_buildings=google_buildings.loc[google_buildings['confidence']>=0.6] # filter out low confidence polygons

Lethoso bbox:  27.011232336601374 -30.67784748254426 29.4573649650311 -28.57059736718119


In [ ]:
# loop through tiles for reclassification
for i in range(len(tile_bboxes)):
    # get tile bbox
    x_min,y_min,x_max,y_max=tile_bboxes.iloc[i]
    print('Processing tile ',i,'with bbox of ',x_min,y_min,x_max,y_max)
    
    # clip land cover maps 2015 and 2021 to tile boundary
    landcover2021_tile=landcover2021.rio.clip_box(minx=x_min,miny=y_min,maxx=x_max,maxy=y_max)
    ds_geobox=landcover2021_tile.geobox # get bbox
    np_landcover2021=landcover2021_tile.squeeze().to_numpy() # data array to numpy array
    np_landcover2021_post=np_landcover2021.copy() # initialise post-processed numpy array
    landcover2015_mapped_tile=xr_reproject(landcover2015_mapped.to_dataset(name='band_data'), ds_geobox, resampling="nearest")
    np_landcover2015_mapped=landcover2015_mapped_tile.to_array().squeeze().to_numpy() # dataset to numpy array
    
    # mode filtering for a smoother classification map
    np_landcover2021_post=modal(np_landcover2021,selem=disk(2.5),mask=np_landcover2021!=0)
#     carry over mining class of previous year as barren
#     np_landcover2021_post[np_landcover2021==13]=12
    # carry over gullies of previous year again as some small areas were smoothed out due to median filtering
    np_landcover2021_post[np_landcover2015_mapped==15]=15
    
    # load and clip hand layer
    hand=xr_reproject(hand, ds_geobox, resampling="average")
    np_hand=hand.to_array().squeeze().to_numpy()
    
    # Make sure water is (only occuring at bottom of watersheds) or fallen within OSM river networks
    river_network_mask=xr_rasterize(gdf=river_network,
                                      da=landcover2015_mapped_tile.to_array().squeeze(),
                                      transform=ds_geobox.transform,
                                      crs=output_crs) # rasterise OSM river network layer
    np_river_network_mask=river_network_mask.to_numpy() # data array to numpy array
    np_landcover2021_post[((np_landcover2021==6)&(np_hand<=45))|(np_river_network_mask==1)]=6 # apply rules
    
    # assign pixels overlapping google building polygons as built-up
    google_buildings_mask=xr_rasterize(gdf=google_buildings,
                                      da=landcover2015_mapped_tile.to_array().squeeze(),
                                      transform=ds_geobox.transform,
                                      crs=output_crs) # rasterise google building layer
    np_google_buildings_mask=google_buildings_mask.to_numpy() # data array to numpy array
    np_landcover2021_post[np_google_buildings_mask==1]=1 # burn in building polygons
    
    # reclassify wetlands around (within 50m of) built-up areas as croplands
    urban_buffered=binary_dilation(np_landcover2021==1,selem=disk(5)) # dilating built-up regions
    np_landcover2021_post[(urban_buffered==1)&(np_landcover2021==7)]=2 # apply rule
    
    # assign pixesl overlapping OSM road network as built-up class
    road_network_mask=xr_rasterize(gdf=road_network,
                                  da=landcover2015_mapped_tile.to_array().squeeze(),
                                  transform=ds_geobox.transform,
                                  crs=output_crs) # # rasterise OSM road network layer
    np_road_network_mask=road_network_mask.to_numpy() # data array to numpy array
    np_landcover2021_post[np_road_network_mask==1]=1 # burn in road polygons
    
    # convert back result back to DataArray
    landcover2021_tile_post=xr.DataArray(data=np_landcover2021_post,dims=['y','x'],coords={'y':landcover2021_tile.y.to_numpy(), 'x':landcover2021_tile.x.to_numpy()})
    landcover2021_tile_post.rio.write_crs(output_crs, inplace=True)
    
    # export as geotiff
    write_cog(landcover2021_tile_post, 'Results/Land_cover2021_postproc_step3_tile_'+str(i)+'.tif', overwrite=True)

In [6]:
# merge into a mosaic once all tiles are processed
! gdal_merge.py -o Results/Land_cover2021_postproc_step3_mosaic.tif -co COMPRESS=Deflate -ot Byte Results/Land_cover2021_postproc_step3_tile_*.tif

0...10...20...30...40...50...60...70...80...90...100 - done.
